In [8]:
#setup environment
import numpy as np 
import pandas as pd 
import os

In [9]:
import matplotlib.pyplot as plt
from PIL import Image

In [10]:
#import data from kaggle database
data_src="/kaggle/input/alzheimer-mri-dataset/Dataset/"

Store image in data array and class label in y array

In [11]:
data=[]
y=[]
for d in os.listdir(data_src):
    for file in os.listdir(data_src+d):
        data.append(Image.open(data_src+d+'/'+file))
        y.append(d)

Convert each image into a numpy array

In [12]:
X=[]
for im in data:
    X.append(np.array(im))

In [13]:
#Convert the list of data into an array
X=np.array(X)

In [14]:
X.shape

In [15]:
#Divide by 255 to standardize the values between 0 and 1
X=X/255

MACHINE LEARNING SETUP

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten
from tensorflow.keras.utils import to_categorical

In [18]:
#count number of each category
non=0
mild=0
mod=0
vm=0
for cat in y:
    if cat=="Mild_Demented":
        mild+=1
    elif cat=="Moderate_Demented":
        mod+=1
    elif cat=="Non_Demented":
        non+=1
    else:
        vm+=1
print("Non Demented: ",non)
print("Very Mild: ",vm)
print("Moderate: ",mod)
print("Mild :",mild)


In [19]:
#change labels from word to number to use to_categorical
#0 will be non, 1 will be very mild, 2 will be mild, 3 will be moderate
y_num=[]
for cat in y:
    if cat=="Mild_Demented":
        y_num.append(2)
    elif cat=="Moderate_Demented":
        y_num.append(3)
    elif cat=="Non_Demented":
        y_num.append(0)
    else:
        y_num.append(1)

In [20]:
#convert to a categorical target as expected by keras
y=to_categorical(y_num)

In [21]:
y.shape

In [22]:
#Reshape to use in a neural network
X=X.reshape(6400,128,128,1)

In [23]:
#Split into a training set and test set
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=25)

In [24]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

In [65]:
#Create the model
cnn=Sequential()
#initially 64 convolution nodes
cnn.add(Conv2D(64,(3,3),padding="same",activation='relu',input_shape=X_train.shape[1:]))
#add a Max Pooling layer
cnn.add(MaxPooling2D())
#another 32 convolution nodes
cnn.add(Conv2D(32,(3,3),padding="same",activation='relu'))
#Add a max pooling
cnn.add(MaxPooling2D())
#Add 32 convolutions
cnn.add(Conv2D(32,(3,3),padding="same",activation='relu'))
#Add a max pooling
cnn.add(MaxPooling2D())
#Flatten before adding fully connected layer
cnn.add(Flatten())
#Add a hidden layer with 100 nodes
cnn.add(Dense(100,activation='relu'))
#Add final output layer with 4 output nodes using softmax
cnn.add(Dense(4,activation='softmax'))

In [66]:
cnn.summary()

In [67]:
cnn.compile(optimizer='adam',loss='categorical_crossentropy',metrics='accuracy')

In [68]:
#Train the Convolutional Neural Network on the data
history=cnn.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test))

In [69]:
#Plot to see how the loss function changed
plt.plot(range(1,26),history.history['loss'],marker="o")
plt.plot(range(1,26),history.history['val_loss'],marker="o")
plt.legend(["Training Loss","Validation Loss"])
plt.xlabel("Epoch")
plt.xticks(range(1,26))
plt.show()

In [71]:
#plot to see how the accuracy changed
plt.plot(range(1,26),history.history['accuracy'],marker="o")
plt.plot(range(1,26),history.history['val_accuracy'],marker="o")
plt.legend(["Training accuracy","Validation accuracy"])
plt.xlabel("Epoch")
plt.xticks(range(1,26))
plt.show()

In [72]:
#Get a prediction from the neural network
y_pred=cnn.predict(X_test)

In [73]:
#The prediction is a sequence of 4 probabilities. We choose the label with the highest probability
y_val=[]
for y in y_pred:
    y_val.append(np.argmax(y))

In [74]:
from sklearn.metrics import confusion_matrix,classification_report

In [75]:
#Convert back to labels 0-3
y_true=[]
for y in y_test:
    y_true.append(np.argmax(y))

In [76]:
#Let's see how the predictions did
print(confusion_matrix(y_true,y_val))

In [77]:
print(classification_report(y_true,y_val))

In [84]:
from sklearn.metrics import accuracy_score

In [83]:
print("Accuracy on test data: ",accuracy_score(y_true,y_val))